In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
df=pd.read_csv(r'C:\Users\rachi\Desktop\NLP\spam.csv', encoding="ISO-8859-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [21]:
df.drop(list(df)[2:], axis = 1, inplace = True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [22]:
df.rename(columns={df.columns[0]:'Label',
                   df.columns[1]:'Message',
                  }
          ,inplace = True)
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# EDA

In [23]:
df.isnull().sum()

Label      0
Message    0
dtype: int64

In [26]:
len(df)

5572

In [28]:
df['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

# Balancing the data

In [31]:
ham=df[df['Label']=='ham']
ham.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...


In [33]:
spam= df[df['Label']=='spam']
spam.head()

,Label,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [35]:
ham.shape, spam.shape

((4825, 2), (747, 2))

In [37]:
ham=ham.sample(spam.shape[0])

In [38]:
ham.shape, spam.shape

((747, 2), (747, 2))

In [41]:
data = ham.append(spam,ignore_index=True)
data.head()

,Label,Message
0,ham,Ok lar i double check wif da hair dresser alre...
1,ham,What you thinked about me. First time you saw ...
2,ham,Only 2% students solved this CAT question in '...
3,ham,I want to send something that can sell fast. ...
4,ham,You could have seen me..i did't recognise you ...


# Data Preparation

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
data.head()

,Label,Message
0,ham,Ok lar i double check wif da hair dresser alre...
1,ham,What you thinked about me. First time you saw ...
2,ham,Only 2% students solved this CAT question in '...
3,ham,I want to send something that can sell fast. ...
4,ham,You could have seen me..i did't recognise you ...


In [64]:
X_train, X_test, y_train, y_test = train_test_split(data['Message'], data['Label'],
                                                    test_size = 0.3,
                                                    random_state = 0, shuffle = True,
                                                   stratify = data['Label'])

## Bag of words creation

In [65]:
vectorizer = TfidfVectorizer()

In [55]:
X_train=vectorizer.fit_transform(X_train)

In [57]:
X_train.shape

(1045, 3613)

## Pipeline and Random Forest

In [66]:
clf = Pipeline([('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier(n_estimators=100, n_jobs=-1))])
clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier(n_jobs=-1))])

In [67]:
y_pred = clf.predict(X_test)

In [69]:
confusion_matrix(y_test,y_pred)

array([[223,   2],
       [ 24, 200]], dtype=int64)

In [70]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.90      0.99      0.94       225
        spam       0.99      0.89      0.94       224

    accuracy                           0.94       449
   macro avg       0.95      0.94      0.94       449
weighted avg       0.95      0.94      0.94       449



In [77]:
accuracy_score(y_test,y_pred)

0.9420935412026726

# SVC

In [105]:
clf = Pipeline([('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, gamma = 'auto'))])
clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, gamma='auto'))])

In [106]:
y_pred = clf.predict(X_test)

In [107]:
confusion_matrix(y_test,y_pred)

array([[220,   5],
       [ 16, 208]], dtype=int64)

In [103]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.94      0.96      0.95       225
        spam       0.96      0.94      0.95       224

    accuracy                           0.95       449
   macro avg       0.95      0.95      0.95       449
weighted avg       0.95      0.95      0.95       449



In [108]:
accuracy_score(y_test,y_pred)

0.9532293986636972